In [ ]:
from scipydirect import minimize
import numpy as np

# Cost function
def cost(x, threats):
    pass

# Circular Threats from paper (page 6)
#                     Centre,    radius
threats = [(np.array([75,  29]),    9   ),
           (np.array([80,  66]),    15  ),
           (np.array([95,  115]),   22.5),
           (np.array([135, 131]),   20  ),
           (np.array([140, 67.5]),  37.5),
           (np.array([197, 125]),   30  ),]


